In [1]:
import warnings
warnings.filterwarnings('ignore')

import nltk
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from transformers import  AutoTokenizer,  AutoModelForSequenceClassification
import datasets

In [2]:
# 데이터 다운로드
# 라벨은 pos:1  neg:0
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews
reviews = [movie_reviews.raw(id) for id in movie_reviews.fileids()]
categoris = [ movie_reviews.categories(id)[0] for id in movie_reviews.fileids() ]
labels = [  1 if label == 'pos' else 0 for label in categoris   ]

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [3]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels \
  = train_test_split(reviews, labels, stratify=labels, test_size=0.2, random_state=42)

In [4]:
# 토크나이져
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
# 훈련/테스트 데이터 토근화
train_encodings = tokenizer(train_texts, truncation=True, padding=True,return_tensors='pt',max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True,return_tensors='pt',max_length=512)
train_encodings['input_ids'].shape,  test_encodings['input_ids'].shape


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(torch.Size([1600, 512]), torch.Size([400, 512]))

In [6]:
# torch dataset 구성
class MovieReviewDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  def __len__(self):
    return len(self.labels)
  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
train_dataset =   MovieReviewDataset(train_encodings, train_labels)
test_dataset =  MovieReviewDataset(test_encodings,test_labels)
print(f'훈련 샘플수 : {len(train_dataset)}')
print(f'테스트 샘플수 : {len(test_dataset)}')

훈련 샘플수 : 1600
테스트 샘플수 : 400


In [8]:
next(iter(train_dataset)).keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])